# Global Crime Statistics - Cleaning
---

With the combined csv files generated/concat/merged from the raw UN data, the goal here is to further clean the csv files into a datafram that will dynamically work with the data collected/cleaned from the client's objectives/conditions for their new resort.
- An important consideration here is that the column headers match the headers from the ideal cities csv file
- THe UN includes the country's name in addition to the 2 and 3 letter code for each contry, however the geoapify only reports the 2 letter country code (ISO2_code)
- Create a list of countries to automatically exclude from crime data (hostile towards tourists, visa difficulties, or travel bans)
- The dataframes generated within this notebook will be added to the existing 'Crime_csv" directory

In [1]:
# Depencancies and Setup
import os
import numpy as np
import pandas as pd


In [2]:
# Create file pathway and create new output directory and filepath
working_dir = os.path.join("..","UN_crime_reports")
crime_data_dir = os.path.join(working_dir,"Crime_csv")

# set up the ideal report city paths here so they can be easily changed if needed
ideal_resort_cities_dir = os.path.join(os.getcwd(),"..","Resources")
ideal_resort_cities_csv = "cities_airports_hotel_lim.csv"

if not os.path.exists(os.path.join(crime_data_dir)):
    os.mkdir(os.path.join(working_dir,"Crime_csv"))



In [3]:
# Remove countryies with active travel bans, countries with highly restrictive visa requirements, or hostile towards tourists
hostile_countries = ["SA","IR"]

In [4]:
# Recall combined UN crime and population CSV
crime_pop_combined = pd.read_csv(os.path.join(working_dir,"Crime_csv","UN_Combined_crime_pop.csv"))

# Remove the index from the previous merge
crime_pop_combined = crime_pop_combined.drop(columns = ['Unnamed: 0'])

# The VALUE column is an object due to the comma separator.  This will cancel any mathematical operations, so....
crime_pop_combined['VALUE'] = crime_pop_combined['VALUE'].str.replace(',','')
crime_pop_combined['VALUE'] = crime_pop_combined['VALUE'].astype(float)
crime_pop_combined = crime_pop_combined.rename(columns = {'VALUE': 'Crimes Committed'})




In [5]:
# Group by county and crime type to get overall number of offences

total_crime_year_county = crime_pop_combined.groupby(['Country','ISO2_code','Year']).sum()["Crimes Committed"]

total_crime_year_county


Country   ISO2_code  Year
Albania   AL         2016    15828.0
                     2017    17261.0
                     2018    20792.0
                     2019    15418.0
                     2020    11019.0
                              ...   
Zambia    ZM         2017     5554.0
                     2018     5421.0
                     2019    10317.0
                     2020     4335.0
Zimbabwe  ZW         2020     3750.0
Name: Crimes Committed, Length: 693, dtype: float64

In [6]:
# Customize the general/combined crime & population dataframe to provide only required information

total_pop_year_country = crime_pop_combined.drop(columns = [
                                                            'Unit of measurement',
                                                            'Crimes Committed',
                                                            'Source',
                                                            'Category',
                                                            'Crime Type'
                                                            ]
                                                            ).drop_duplicates()

total_pop_year_country



,Country,Year,ISO3_code,ISO2_code,PopTotal
0,Albania,2016,ALB,AL,2881.063
15,Albania,2017,ALB,AL,2879.355
30,Albania,2018,ALB,AL,2877.013
45,Albania,2019,ALB,AL,2873.883
60,Albania,2020,ALB,AL,2866.849
...,...,...,...,...,...
28856,Iraq (Central Iraq),2018,NaN,NaN,NaN
30556,Timor-Leste,2016,TLS,TL,1224.562
30559,Timor-Leste,2017,TLS,TL,1243.235
30707,South Africa,2016,ZAF,ZA,56422.274


In [7]:
# Considering the API pulls, all of the countries are listed by ISO2_code, so I these df will be merged based on this code
# while keeping the country name for reference.  Also, I choose how = 'inner' because I need these two df to be the same size after merge
# to avoid NaN, null, or blank values. 
# There could still be some special districts that slipped though the initial data collection

total_crime_pop = pd.merge(total_crime_year_county,total_pop_year_country, how = 'inner',on='ISO2_code')
total_crime_pop.head()

,ISO2_code,Crimes Committed,Country,Year,ISO3_code,PopTotal
0,AL,15828.0,Albania,2016,ALB,2881.063
1,AL,15828.0,Albania,2017,ALB,2879.355
2,AL,15828.0,Albania,2018,ALB,2877.013
3,AL,15828.0,Albania,2019,ALB,2873.883
4,AL,15828.0,Albania,2020,ALB,2866.849


In [8]:
# check for NaN, Null, or blank values... That is find all of the special regions/ UN districts that slipped through!!!
def nan_null_test(df):
    null_mask = df.isnull()
    nan_mask = df.isna()
    result = df.loc[null_mask.any(axis=1)]
    print(f"Number of NaN values : {len(result)}")
    
    nan_mask = df.isna()
    result = df.loc[nan_mask.any(axis=1)]
    print(f"Number of NaN values : {len(result)}")
    return(result)

nan_null_test(total_crime_pop)

Number of NaN values : 0
Number of NaN values : 0


,ISO2_code,Crimes Committed,Country,Year,ISO3_code,PopTotal


In [9]:
# Calculate crimes committed per 100,000 population.
# Keep in mind that values in PopTotal have already been divided by 1000, i.e. PopTotal = population/1000 citizens
total_crime_pop['Crimes/ 100,000 population'] = 100 * total_crime_pop['Crimes Committed'] / total_crime_pop['PopTotal']

# Change crime/population column names to match ideal cities csv from API requests.
total_crime_pop = total_crime_pop.rename(columns={'Country':'Country Name','ISO2_code':'Country'})
total_crime_pop.head()

,Country,Crimes Committed,Country Name,Year,ISO3_code,PopTotal,"Crimes/ 100,000 population"
0,AL,15828.0,Albania,2016,ALB,2881.063,549.380558
1,AL,15828.0,Albania,2017,ALB,2879.355,549.706445
2,AL,15828.0,Albania,2018,ALB,2877.013,550.153927
3,AL,15828.0,Albania,2019,ALB,2873.883,550.753110
4,AL,15828.0,Albania,2020,ALB,2866.849,552.104418


In [10]:
# Import ideal cities CSV cleaned from the API requests and client's requirements

ideal_cities_list = pd.read_csv(os.path.join(ideal_resort_cities_dir,ideal_resort_cities_csv))
ideal_cities_df = ideal_cities_list.drop(columns = [
    'City_ID', 
    'Max Temp', 
    'Humidity',
    'Cloudiness',
    'Wind Speed', 
    'Date',
    'Rank',
    'Crime Index',
    'Safety Index',
    'Number of Hotels',
    'Number of Resturants',
    'Bodies of Water',
    'Tourist Attractions',
    'Natural Places',
    'Hospital',
    'Entertainment',
    'Rental Car',
    'Airport'
])

# Remove hostiile countries
ideal_cities_df = ideal_cities_df[~ideal_cities_df['Country'].isin(hostile_countries)].set_index('Country')

# ideal_cities_df.head(50)

In [11]:
# Merge the ideal cities list with the global crime df and remove all rows that do not match...

crime_comparo_ideal_cities = pd.merge(ideal_cities_df, total_crime_pop, how = 'inner', on='Country')
crime_comparo_ideal_cities = crime_comparo_ideal_cities.sort_values(by = "Country",ascending = True)


In [12]:
# test for NaN null
nan_null_test(crime_comparo_ideal_cities)

Number of NaN values : 0
Number of NaN values : 0


,Country,City,Lat,Lng,Crimes Committed,Country Name,Year,ISO3_code,PopTotal,"Crimes/ 100,000 population"


In [13]:
# Create an empty dataframe for each country and it's normalized (per 100k people) crimes
ideal_city_crime_risk = pd.DataFrame(columns = ['Country', "Year",'Crimes/100,000 people'])

#Define which countries will be represneted in the ideal cities dataframe, and create a list to filter out unwanted countries
unique_countries = crime_comparo_ideal_cities['Country'].unique()

# filter the overall ideal city's crime
sorted_ideal_city_crime = crime_comparo_ideal_cities[crime_comparo_ideal_cities['Country'].isin(unique_countries)]

# Populate the empty dataframe and include only the columns of interest
ideal_city_crime_risk=sorted_ideal_city_crime[['Country' , 'Year' , 'Crimes/ 100,000 population']]


In [14]:
# Generate values of high summary table
country_name = crime_comparo_ideal_cities[crime_comparo_ideal_cities['Country'].isin(unique_countries)]['Country Name'].unique()
country_code = ideal_city_crime_risk['Country'].unique()
crime_reported_years = ideal_city_crime_risk.groupby('Country')['Year'].nunique()
latest_crime_report = ideal_city_crime_risk.groupby(['Country']).max()['Year']
mean_country_crime_over_year = ideal_city_crime_risk.groupby(['Country']).mean()['Crimes/ 100,000 population'].round(2)
num_cities_in_country = crime_comparo_ideal_cities.groupby(['Country']).count()['City']
oldest_crime_report = ideal_city_crime_risk.groupby(['Country']).min()['Year']


In [15]:
# Generage high level overall country crime report csv file
city_crime_summary = pd.DataFrame({
    "Country Name": country_name,
    "Country Code": country_code,
    "Number of Cities Meeting Criteria": num_cities_in_country,
    "Most Recent Crime Report": latest_crime_report,
    "Oldest Crime Report Included": oldest_crime_report,
    "Number of Annual Crime Reports Available": crime_reported_years,
    "Average Crimes Committed /100,000 People":mean_country_crime_over_year
})

city_crime_high_summary = city_crime_summary.reset_index(drop=True)
city_crime_high_summary = city_crime_summary.sort_values(by="Average Crimes Committed /100,000 People", ascending = True)
city_crime_high_summary.to_csv(os.path.join(crime_data_dir,"ideal_country_crime_high_summary.csv"))
city_crime_high_summary

,Country Name,Country Code,Number of Cities Meeting Criteria,Most Recent Crime Report,Oldest Crime Report Included,Number of Annual Crime Reports Available,"Average Crimes Committed /100,000 People"
Country,,,,,,,
MG,Madagascar,MG,4,2018,2017,2,0.34
BH,Bahrain,BH,36,2021,2016,6,3.08
EG,Egypt,EG,16,2020,2016,4,13.57
LY,Libya,LY,4,2017,2016,2,15.18
BR,Brazil,BR,150,2020,2016,5,1808.45
US,United States of America,US,288,2021,2016,6,1932.91
AR,Argentina,AR,75,2020,2016,5,1987.46
CL,Chile,CL,25,2020,2016,5,3350.20
UY,Uruguay,UY,125,2020,2016,5,5306.52


In [17]:
# Generate pivot table to seek trends in the crime rate, 
# indexing the crime category and reported crime tpyes per country, per year reported
# https://builtin.com/data-science/pandas-pivot-tables

annual_reported_normalized_crime = np.round(pd.pivot_table(
    crime_comparo_ideal_cities,
    values = 'Crimes/ 100,000 population',
    index = 'Country Name',
    columns = 'Year',
    aggfunc=np.mean
    ),
    2)


# Convert NaN with blank values for future plotting
annual_reported_normalized_crime.fillna('', inplace = True)

# Create CSV file
annual_reported_normalized_crime.to_csv(os.path.join(crime_data_dir,"annual_reported_crime.csv"))
annual_reported_normalized_crime

Year,2016,2017,2018,2019,2020,2021
Country Name,,,,,,
Argentina,2019.77,2002.06,1985.88,1971.15,1958.43,
Bahrain,3.2,3.10,3.04,3.02,3.06,3.09
Brazil,1836.59,1822.09,1807.69,1793.89,1782.0,
Chile,3462.2,3408.53,3347.86,3288.42,3243.98,
Egypt,14.02,13.75,13.49,,13.02,
Libya,15.3,15.07,,,,
Madagascar,,0.35,0.34,,,
United States of America,1965.33,1949.95,1936.16,1923.53,1914.24,1908.25
Uruguay,5322.57,5309.45,5301.95,5299.83,5298.79,


## Drill Down to Consider the Types of Crimes Committed Within the Countries on the Ideal Resort Cities List



In [43]:
#  Merge the ideal cities list with the global crime df and remove all rows that do not match...
#  crime/population column names to match ideal cities csv from API requests.
crime_pop_combined = crime_pop_combined.rename(columns={'Country':'Country Name','ISO2_code':'Country'})

crime_pop_combined

tot_crime_comparo_ideal_cities = pd.merge(ideal_cities_df, crime_pop_combined, how = 'inner', on='Country')
tot_crime_comparo_ideal_cities = tot_crime_comparo_ideal_cities.sort_values(by = "Country",ascending = True)
tot_crime_comparo_ideal_cities = tot_crime_comparo_ideal_cities.drop(columns = [
                                                            'Unit of measurement',
                                                            'City',
                                                            'Lat',
                                                            'Lng',
                                                            'Source',
                                                            'ISO3_code'
                                                            ]
                                                            )

tot_crime_comparo_ideal_cities.groupby(['Country Name', 'Year', 'Crime Type','Category']).count()['Crimes Committed']

Country Name  Year  Crime Type                Category                       
Argentina     2016  Economic                  Theft                              3
                    Firearms                  Assembled arms                     3
                                              Converted arms                     3
                                              Industrially manufactured arms     3
                                              Modified arms                      3
                                                                                ..
Uruguay       2020  Violent & Sexual Assault  Kidnapping                         5
                                              Serious assault                    5
                                              Sexual violence                    5
                                              Sexual violence: Rape              5
                                              Sexual violence: Sexual assault    5
Name: Cri

In [44]:
# Create pivot table comparing the number of crimes committed by each country, and crime category, per year
tot_crime_type_pivot = np.round(pd.pivot_table(
    tot_crime_comparo_ideal_cities,
    values = 'Crimes Committed',
    index = ['Country Name','Crime Type'],
    columns = 'Year',
    aggfunc=np.mean
    ),2)

# Convert NaN with blank values for future plotting
tot_crime_type_pivot.fillna('', inplace = True)
tot_crime_type_pivot.to_csv(os.path.join(crime_data_dir,"tot_crime_type_pivot.csv"))
tot_crime_type_pivot

Year                                                     2016        2017  \
Country Name             Crime Type                                         
Argentina                Economic                    270125.0    127577.0   
                         Firearms                     3212.56     3874.51   
                         Homicide                       146.0      379.43   
                         Human Trafficking              86.08      207.17   
                         Violent & Sexual Assault    120984.4    96042.33   
Bahrain                  Human Trafficking               21.0        16.5   
Brazil                   Drug Related                            244583.0   
                         Economic                     78783.0    664806.0   
                         Firearms                      783.47      503.93   
                         Homicide                                           
                         Human Trafficking               42.0               
                         Violent & Sexual Assault    447176.2    463138.6   
Chile                    Drug Related                 26040.0     34619.0   
                         Economic                    62604.25    27043.41   
                         Firearms                                           
                         Homicide                      100.29      124.97   
                         Human Trafficking                6.5               
                         Violent & Sexual Assault    21821.57    13251.33   
Egypt                    Drug Related                                       
                         Human Trafficking               25.0        18.9   
Libya                    Firearms                       53.06       54.69   
                         Human Trafficking              198.0               
Madagascar               Drug Related                               149.0   
                         Human Trafficking                           11.5   
United States of America Drug Related                             42155.0   
                         Economic                  1346179.33  1304542.83   
                         Firearms                     40837.0     45429.0   
                         Homicide                     2612.77     2539.47   
                         Human Trafficking             749.43       887.3   
                         Violent & Sexual Assault   422733.33    422195.0   
Uruguay                  Drug Related                   995.0       947.0   
                         Economic                     32996.5     33932.5   
                         Firearms                     1454.33       648.5   
                         Homicide                       46.92       43.11   
                         Human Trafficking               89.0        72.0   
                         Violent & Sexual Assault      4072.5      4033.0   

Year                                                     2018        2019  \
Country Name             Crime Type                                         
Argentina                Economic                    139016.5    167431.0   
                         Firearms                                           
                         Homicide                       391.5      386.04   
                         Human Trafficking             195.62       237.8   
                         Violent & Sexual Assault    99008.67   110863.17   
Bahrain                  Human Trafficking               12.0        13.0   
Brazil                   Drug Related                289395.0               
                         Economic                   826042.33   638818.33   
                         Firearms                    13320.71               
                         Homicide                      830.67       723.0   
                         Human Trafficking               50.5        46.5   
                         Violent & Sexual Assault    416450.0    368279.

In [53]:
# Create more detailed pivot table comparing the number of crimes committed by each country, crime category and crime type per year
tot_crime_category_pivot = np.round(pd.pivot_table(
    tot_crime_comparo_ideal_cities,
    values = 'Crimes Committed',
    index = ['Country Name', 'Crime Type','Category'],
    columns = 'Year',
    aggfunc=np.count_nonzero
    ),2)

# Convert NaN with blank values for future plotting
tot_crime_category_pivot.fillna('', inplace = True)
tot_crime_category_pivot.to_csv(os.path.join(crime_data_dir,"tot_crime_category_pivot.csv"))
tot_crime_category_pivot

Year                                                                                     2016  \
Country Name Crime Type               Category                                                  
Argentina    Economic                 Theft                                               3.0   
                                      Trade or possession of protected or prohibited ...        
             Firearms                 Assembled arms                                      0.0   
                                      Converted arms                                      0.0   
                                      Industrially manufactured arms                      3.0   
...                                                                                       ...   
Uruguay      Violent & Sexual Assault Robbery                                             5.0   
                                      Serious assault                                     5.0   
                                      Sexual violence                                     5.0   
                                      Sexual violence: Rape                               5.0   
                                      Sexual violence: Sexual assault                     5.0   

Year                                                                                     2017  \
Country Name Crime Type               Category                                                  
Argentina    Economic                 Theft                                               3.0   
                                      Trade or possession of protected or prohibited ...  3.0   
             Firearms                 Assembled arms                                      0.0   
                                      Converted arms                                      0.0   
                                      Industrially manufactured arms                      3.0   
...                                                                                       ...   
Uruguay      Violent & Sexual Assault Robbery                                             5.0   
                                      Serious assault                                     5.0   
                                      Sexual violence                                     5.0   
                                      Sexual violence: Rape                               5.0   
                                      Sexual violence: Sexual assault                     5.0   

Year                                                                                     2018  \
Country Name Crime Type               Category                                                  
Argentina    Economic                 Theft                                               3.0   
                                      Trade or possession of protected or prohibited ...  3.0   
             Firearms                 Assembled arms                                            
                                      Converted arms                                            
                                      Industrially manufactured arms                            
...                                                                                       ...   
Uruguay      Violent & Sexual Assault Robbery                                             5.0   
                                      Serious assault                                     5.0   
                                      Sexual violence                                     5.0   
                                      Sexual violence: Rape                               5.0   
                                      Sexual violence: Sexual assault                     5.0   

Year                                                                                     2019  \
Country Name Crime Type               Category                                                  
Argentina    Economi

/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_52649/1354934667.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  tot_crime_types_list_pivot = pd.pivot_table(


Crime Type
Country Name
Argentina
Bahrain
Brazil
Chile
Egypt
Libya
Madagascar
United States of America
Uruguay
